In [1]:
import matplotlib
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import os
import umap
import datashader as ds
import colorcet as cc
import igraph
import tqdm
from scipy import sparse
from scipy import stats
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import LatentDirichletAllocation
from statsmodels.stats.multitest import multipletests
from sklearn.cluster import KMeans
from scipy.spatial import Delaunay
import json
import itertools
import pynndescent
import time

from matplotlib.collections import PolyCollection
from matplotlib.colors import ListedColormap

from dredFISH.Analysis import TissueGraph
from dredFISH.Visualization import Viz
from dredFISH.Utils.__init__plots import * 
from dredFISH.Utils import powerplots
from dredFISH.Utils import miscu
from dredFISH.Utils import tmgu

import importlib
importlib.reload(Viz)
importlib.reload(TissueGraph)

<module 'dredFISH.Analysis.TissueGraph' from '/bigstore/GeneralStorage/fangming/projects/dredfish/packages/dredFISH/dredFISH/Analysis/TissueGraph.py'>

#### Load data

In [2]:
respath = '/bigstore/GeneralStorage/fangming/projects/dredfish/figures/'

In [3]:
basepth = '/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t5'
!ls -alhtr $basepth
!head $basepth"/TMG.json"

total 85M
drwxrwxrwx 9 zach     wollmanlab 4.0K Jul 20 10:38 ..
lrwxrwxrwx 1 fangming wollmanlab   67 Jul 20 10:38 DPNMF_PolyA_2021Nov19_Section_-1850X_270Y_matrix.csv -> ../Dataset1-t4/DPNMF_PolyA_2021Nov19_Section_-1850X_270Y_matrix.csv
lrwxrwxrwx 1 fangming wollmanlab   69 Jul 20 10:38 DPNMF_PolyA_2021Nov19_Section_-1850X_270Y_metadata.csv -> ../Dataset1-t4/DPNMF_PolyA_2021Nov19_Section_-1850X_270Y_metadata.csv
-rw-r--r-- 1 fangming wollmanlab   32 Jul 20 10:39 README.txt
-rw-r--r-- 1 fangming wollmanlab   88 Jul 20 10:41 TMG.json
-rw-r--r-- 1 fangming wollmanlab  53M Jul 20 10:41 cell.h5ad
drwxr-xr-x 3 fangming wollmanlab 4.0K Jul 20 10:45 .
-rw-r--r-- 1 fangming wollmanlab  32M Jul 20 10:45 default_analysis.csv
drwxr-xr-x 4 fangming wollmanlab 4.0K Jul 21 11:49 figures
{"layers_graph": [], "layer_taxonomy_mapping": {}, "Taxonomies": [], "Layers": ["cell"]}

In [4]:
df = pd.read_csv(
    os.path.join(basepth, "default_analysis.csv"), index_col=0)
df

,x,y,x2,y2,hemi,b0,b1,b2,b3,b4,...,type_r0.01,type_r0.02,type_r0.05,type_r0.1,type_r0.2,type_r0.5,type_r1,type_r2,type_r5,type_r10
0,1911.666962,-1749.638192,1833.366541,-1831.521636,1,-0.603849,1.714861,0.594060,0.139496,1.509942,...,t1,t1,t1,t3,t2,t4,t3,t16,t47,t40
1,1850.678429,-1747.172043,1772.544083,-1826.392145,1,-0.530279,1.681959,1.301185,-0.464006,1.570054,...,t1,t1,t1,t3,t2,t4,t3,t16,t47,t40
2,1515.123295,-1746.940031,1437.319766,-1811.493848,1,-0.603849,-0.362498,0.620053,-1.128423,1.206578,...,t1,t1,t1,t3,t2,t10,t9,t20,t24,t66
3,1426.161788,-1748.456394,1348.376999,-1809.120415,1,-0.298989,1.023065,1.517082,-0.341796,0.475449,...,t1,t1,t1,t3,t2,t10,t9,t20,t51,t121
4,1739.792930,-1749.407386,1661.666850,-1823.778747,1,-0.603849,-1.351786,1.025515,-1.128423,0.327118,...,t1,t1,t1,t1,t1,t1,t2,t3,t34,t21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77841,-2188.756848,-1084.480934,-2234.065840,-987.777939,0,-0.603849,-0.271399,-0.666686,0.269214,0.821624,...,t1,t1,t1,t3,t2,t8,t18,t36,t56,t57
77842,-2257.822983,-1083.270665,-2303.013073,-983.550072,0,0.936062,0.386319,-0.614982,1.752306,0.067531,...,t1,t1,t3,t4,t6,t8,t10,t6,t1,t73
77843,-2475.320577,-1083.995803,-2520.334508,-974.768090,0,-0.603849,-0.228163,-0.612215,0.895132,0.247026,...,t1,t1,t2,t2,t5,t6,t7,t25,t41,t82
77844,-2311.578984,-1086.717336,-2356.868349,-984.643871,0,-0.603849,-1.153599,0.699586,-0.204732,1.546018,...,t1,t1,t1,t1,t1,t1,t1,t5,t12,t26


In [5]:
TMG = TissueGraph.TissueMultiGraph(basepath=basepth, 
                                   redo=False, # load existing 
                                  )

/bigstore/GeneralStorage/Data/dredFISH/Dataset1-t5/cell.h5ad


In [6]:
# spatial coordinates
layer = TMG.Layers[0]
XY = layer.XY
x, y = XY[:,0], XY[:,1]
###
x, y = y, x # a temporary hack
###

cells = layer.adata.obs.index.values

N = layer.N
# measured basis
ftrs_mat = layer.feature_mat

# umap_mat = umap.UMAP(n_neighbors=30, min_dist=0.1).fit_transform(ftrs_mat)




# Lateral symmetry measure

In [41]:
def build_feature_graph_knnlite(ftrs_mat, k=15, metric='cosine'):
    """
    """
    N = len(ftrs_mat)
    
    # kNN graph
    knn = pynndescent.NNDescent(ftrs_mat,
                                n_neighbors=k,
                                metric=metric,
                                diversify_prob=1,
                                pruning_degree_multiplier=1.5,
                                )
    idx, _ = knn.neighbor_graph

    # to adj and to graph
    i = np.repeat(np.arange(N), k-1)
    j = idx[:,1:].reshape(-1,)
    adj_mat = sparse.coo_matrix((np.repeat(1, len(i)), (i,j)), shape=(N,N))
    G = tmgu.adjacency_to_igraph(adj_mat, directed=False, simplify=True)
    
    return G

In [42]:
# from meta
f = '/bigstore/GeneralStorage/fangming/projects/dredfish/data_dump/analysis_meta_Mar31.json'
with open(f, 'r') as fh:
    meta = json.load(fh)

In [64]:
df_h1 = df[df['hemi']==0].copy()
df_h2 = df[df['hemi']==1].copy()

# separate cells h1 and h2
cells_h1 = df.index[df['hemi'] == 0].values
cells_h2 = df.index[df['hemi'] == 1].values

In [44]:
%%time
ftrs_mat_h1 = ftrs_mat[cells_h1]
G_h1 = build_feature_graph_knnlite(ftrs_mat_h1, k=15, metric='cosine')

ftrs_mat_h2 = ftrs_mat[cells_h2]
G_h2 = build_feature_graph_knnlite(ftrs_mat_h2, k=15, metric='cosine')

CPU times: user 1min 6s, sys: 57.6 s, total: 2min 4s
Wall time: 10.5 s


In [65]:
# clustering half-and-half
resolutions = [0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1,2,5,10]
clst_mat_h1 = []
clst_mat_h2 = []
for i, r in tqdm.tqdm(enumerate(resolutions)):
    #
    types_h1 = miscu.leiden(G_h1, cells_h1, resolution=r)
    types_h2 = miscu.leiden(G_h2, cells_h2, resolution=r)
    
    # add to a df 
    df_h1[f'type_r{r}'] = types_h1
    df_h2[f'type_r{r}'] = types_h2
    clst_mat_h1.append(types_h1)
    clst_mat_h2.append(types_h2)
    
clst_mat_h1 = np.array(clst_mat_h1)
clst_mat_h2 = np.array(clst_mat_h2)

10it [00:19,  1.99s/it]


In [66]:
df_h1

,x,y,x2,y2,hemi,b0,b1,b2,b3,b4,...,type_r0.01,type_r0.02,type_r0.05,type_r0.1,type_r0.2,type_r0.5,type_r1,type_r2,type_r5,type_r10
722,-2853.511340,-1748.049367,-2927.188428,-1621.657011,0,0.289528,0.918979,-0.458898,1.464897,0.152791,...,1,1,3,4,4,5,4,1,5,8
723,-2981.578470,-1750.208045,-3055.227522,-1618.216044,0,-0.603849,-1.733295,-1.721907,-1.128423,5.098941,...,1,1,1,1,1,1,7,5,35,86
724,-3133.422500,-1748.206369,-3206.838951,-1609.579453,0,0.031336,0.066740,-0.576144,2.661681,-0.357316,...,1,2,2,2,5,6,9,29,42,20
725,-3010.954956,-1752.105725,-3084.658878,-1618.827917,0,-0.603849,-1.733295,-1.798620,-1.128423,1.886831,...,1,1,1,1,1,3,1,14,8,59
726,-2791.064781,-1754.223207,-2865.071395,-1630.554377,0,-0.603849,-0.938579,-0.588792,-1.128423,2.884774,...,1,1,1,1,1,1,7,5,35,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77841,-2188.756848,-1084.480934,-2234.065840,-987.777939,0,-0.603849,-0.271399,-0.666686,0.269214,0.821624,...,1,1,1,4,4,5,4,21,30,84
77842,-2257.822983,-1083.270665,-2303.013073,-983.550072,0,0.936062,0.386319,-0.614982,1.752306,0.067531,...,1,1,3,4,4,5,4,1,9,21
77843,-2475.320577,-1083.995803,-2520.334508,-974.768090,0,-0.603849,-0.228163,-0.612215,0.895132,0.247026,...,1,2,2,2,5,6,9,9,50,54
77844,-2311.578984,-1086.717336,-2356.868349,-984.643871,0,-0.603849,-1.153599,0.699586,-0.204732,1.546018,...,1,1,1,1,1,3,1,14,35,27


In [68]:
for i, r in enumerate(resolutions):
    hue = f'type_r{r}'
    output = None
    powerplots.plot_type_spatial_umap(df_h1, hue, output=output)

TypeError: plot_type_spatial_umap() got an unexpected keyword argument 'x'